# Read Data

In [0]:
import glob
import os
import numpy as np
import csv
import cv2
import io
from sklearn.preprocessing import StandardScaler
import shutil

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [0]:
#Move files into training and validation directory.
trainfile = open('/home/nesl/swapnil/trainfileslist.csv', newline='')
reader = csv.reader(trainfile)
for row in reader:
    curFile = row[0]
    sourcepathA = '/home/nesl/swapnil/data/left_hip/'+curFile+'.csv'
    sourcepathB = '/home/nesl/swapnil/data/right_arm/'+curFile+'.csv'
    sourcepathC = '/home/nesl/swapnil/data/right_wrist/'+curFile+'.csv'
    destpathA = '/home/nesl/swapnil/data_2/train/left_hip/'+curFile+'.csv'
    destpathB = '/home/nesl/swapnil/data_2/train/right_arm/'+curFile+'.csv'
    destpathC = '/home/nesl/swapnil/data_2/train/right_wrist/'+curFile+'.csv'
    shutil.copy(sourcepathA,destpathA)
    shutil.copy(sourcepathB,destpathB)
    shutil.copy(sourcepathC,destpathC)
    
valfile = open('/home/nesl/swapnil/valfileslist.csv', newline='')
reader = csv.reader(valfile)
for row in reader:
    curFile = row[0]
    sourcepathA = '/home/nesl/swapnil/data/left_hip/'+curFile+'.csv'
    sourcepathB = '/home/nesl/swapnil/data/right_arm/'+curFile+'.csv'
    sourcepathC = '/home/nesl/swapnil/data/right_wrist/'+curFile+'.csv'
    destpathA = '/home/nesl/swapnil/data_2/val/left_hip/'+curFile+'.csv'
    destpathB = '/home/nesl/swapnil/data_2/val/right_arm/'+curFile+'.csv'
    destpathC = '/home/nesl/swapnil/data_2/val/right_wrist/'+curFile+'.csv'
    shutil.copy(sourcepathA,destpathA)
    shutil.copy(sourcepathB,destpathB)
    shutil.copy(sourcepathC,destpathC)   

In [0]:
data_dir_train='/home/nesl/swapnil/data_2/train'
data_dir_val= '/home/nesl/swapnil/data_2/val'

In [0]:
sub_dirs = ['left_hip','right_arm','right_wrist' ] #three sensors, 9 channels total

In [0]:
def parse_IMU_files(parent_dir, sub_dirs, startTime, endTime, file_name, window_length):
    
    data = []
    for sub_dir in sub_dirs:
        channel=[]
        
        for fn in glob.glob(os.path.join(parent_dir,sub_dir, file_name)):
            file = open(fn, newline='')
            reader = csv.reader(file)
            first = True
            count = 0
            for row in reader:
                
                if first:
                    first = False
                    continue
                
                timestamp=float(row[3]) #4th column is timestamp
                if timestamp >=startTime and timestamp <=endTime and count<window_length:
                    
                    channel.append([float(row[0]),float(row[1]),float(row[2])])
                    count = count + 1  
                    
        data.append(channel)         
    return data

In [0]:
def get_train_data(data_dir, sub_dirs):
    files = os.listdir(data_dir+'/left_hip')
    number_of_samples = 300
    
    labels_macro =dict()
    
    labels_macro['sandwich'] = 0
    labels_macro['fruitsalad'] = 1
    labels_macro['cereal'] = 2
    
    
    #read the labels
    labels_loc = '/home/nesl/Downloads/LabelTable.csv'
    file_label = open(labels_loc, newline='')
    label_reader = csv.reader(file_label)
    file_label_mapping = dict()
    
    for row in label_reader:
        file_label_mapping[row[0]+'.csv'] = labels_macro[row[1]]
        
    all_data = []
    all_labels = []
    for f in files:
        
        st_index = 0
        end_index = 30000
        step = 1000 #overlapping window, step: 1000. 
        window_index = 6000 #6 second window
        
        print('reading file:',f)
        f_name = f
        
        curr_label_file = file_label_mapping[f_name]
        
        while st_index+step < end_index:
        
            data = parse_IMU_files(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            train_data_sample  = np.zeros((9, number_of_samples))
            train_data_label   = curr_label_file
            for i in range(len(data)):
                for j in range(len(data[i])):
                    train_data_sample[i*3,j]=data[i][j][0]
                    train_data_sample[i*3+1,j]=data[i][j][1]
                    train_data_sample[i*3+2,j]=data[i][j][2]
            
            all_data.append(train_data_sample)
            all_labels.append(train_data_label)
            
            st_index = st_index+step
    
    return all_data, all_labels

In [0]:
#receive windowed training and validation data
train_x, train_y = get_train_data(data_dir_train,sub_dirs)
val_x, val_y = get_train_data(data_dir_val,sub_dirs)

In [0]:
train_samples = np.array(train_x) 
train_labels2 = np.array(train_y)
val_samples = np.array(val_x) 
val_labels2 = np.array(val_y)

#Uncomment the lines below for normalization
#Optional: normalization
#scalers = {}

# for i in range(train_samples.shape[1]):
#     scalers[i] = StandardScaler()
#     train_samples[:,i,:] = scalers[i].fit_transform(train_samples[:,i,:])
    
# for i in range(val_samples.shape[1]):
#     val_samples[:,i,:] = scalers[i].transform(val_samples[:,i,:])

In [0]:
print(train_samples.shape)
print(train_labels2.shape)
print(val_samples.shape)
print(val_labels2.shape)

In [0]:
#convert to one hot encoding
from keras.utils  import to_categorical
train_labels = to_categorical(train_labels2)
train_labels.shape
val_labels = to_categorical(val_labels2)
val_labels.shape

In [0]:
print(train_samples.shape)
print(train_labels.shape)
print(val_samples.shape)
print(val_labels.shape)

In [0]:
#For CNN and BidirLSTM:
train_samples = train_samples.reshape((-1, 9,number_of_samples, 1))
val_samples = val_samples.reshape((-1, 9,number_of_samples, 1))
#For LSTM, uncomment these lines and comment above two lines
#train_samples = train_samples.reshape((-1, number_of_samples, 9))
#val_samples = val_samples.reshape((-1, number_of_samples, 9))

#9 refers to 9 accelerometer channels

# Models

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Permute, Reshape
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [0]:
def _data_reshaping(X_tr, X_va, X_ts, network_type):
    _, win_len, dim = X_tr.shape
    print(network_type)
    if network_type=='CNN' or network_type=='ConvLSTM':

        X_tr = np.swapaxes(X_tr,1,2)
        X_va = np.swapaxes(X_va,1,2)
        X_ts = np.swapaxes(X_ts,1,2)

        X_tr = np.reshape(X_tr, (-1, dim, win_len, 1))
        X_va = np.reshape(X_va, (-1, dim, win_len, 1))
        X_ts = np.reshape(X_ts, (-1, dim, win_len, 1))
    
    return X_tr, X_va, X_ts

In [0]:
def model_variant(model, num_feat_map, dim, network_type,p):
    print(network_type)
    if network_type == 'ConvLSTM':
        model.add(Permute((2, 1, 3))) 
        model.add(Reshape((-1,num_feat_map*dim)))
        model.add(Bidirectional(LSTM(128, return_sequences=False, stateful=False)))
    if network_type == 'CNN':
        
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization()) 
        model.add(Dropout(p))

        
def model_conv(model, num_feat_map,p,b):
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10),    
                 activation='relu',
                 input_shape=(dim, win_len, 1),
                 padding='same'))
    
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same'))
    
    if (b==1):
        model.add(BatchNormalization()) 
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same'))
    
    if (b==1):
        model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(1, 3)))
    
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same')) 
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same'))
    if (b==1):
        model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(p))
    
    model.add(Conv2D(num_feat_map, kernel_size=(1, 10), activation='relu',padding='same'))  
    if (b==1):
        model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(1, 2)))
    
    model.add(Dropout(p))
    
def model_LSTM(model,p):
    model.add(LSTM(num_hidden_lstm, 
               input_shape=(win_len,dim), 
               return_sequences=True))
    model.add(Dropout(p))
    model.add(LSTM(num_hidden_lstm, return_sequences=False))
    model.add(Dropout(p))
    
def model_output(model):
    model.add(Dense(num_classes, activation='softmax'))

In [0]:
batch_size = 64
num_feat_map = 128
num_hidden_lstm = 128
num_classes = 3

#Uncomment the classifier you are working with:
#network_type = 'CNN'
network_type = 'ConvLSTM'
#network_type = 'LSTM'
#for CNN/DCBL
_, dim, win_len,_ = train_samples.shape
#for LSTM uncomment the following line and comment above line
#_, dim, win_len = train_samples.shape

print(win_len)
print(dim)

In [0]:
p=0.5 #Dropout
b = 1 #BatchNorm
print('building the model ... ')
model = Sequential()

if network_type=='CNN' or network_type=='ConvLSTM':
    model_conv(model, num_feat_map,p,b)
    model_variant(model, num_feat_map, dim, network_type,p)
if network_type=='LSTM':
    model_LSTM(model,p)
       
model_output(model)    
model.summary()

In [0]:
X_train = train_samples
y_train = train_labels
X_val = val_samples
y_val = val_labels

In [0]:
print(X_train.shape, X_val.shape)

In [0]:
epochs = 200

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

# checkpoint
filepath="DCBL_6_NoMocap.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

H = model.fit(train_samples, train_labels,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            shuffle=True,
            validation_data=(X_val, y_val),
             callbacks=callbacks_list
             )

In [0]:
history = H

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
print(np.argmax(np.array(history.history['val_acc'])))
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
print('Maximum validation accuracy: ',np.max(np.array(history.history['val_acc'])))
print('Training accuracy of best model: ',np.array(history.history['acc'])[np.argmax(np.array(history.history['val_acc']))])
